In [1]:
import requests
import json
import datetime as dt
import os
import concurrent.futures as futures
from tqdm.notebook import tqdm

In [2]:
urlFmt = "https://dapi.kakao.com/v2/search/image?query={query}&page={page}&size={size}"


In [3]:
query = "python"
page = 1
size = 50
key = ""

In [5]:
session = requests.Session()

session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
    "Authorization": "KakaoAK %s" % key
})

In [6]:
url = urlFmt.format(query=query, page=page, size=size)

try:
    r = session.get(url)

    if r.status_code !=200:
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        raise Exception(msg)

except Exception as e:
    print("접속에 실패했습니다.")
    print(e)  

In [14]:
page = 1
isEnd = False

mydata = []

while not isEnd:
    url = urlFmt.format(query=query, page=page, size=size)
    #print(url)

    try:
        r = session.get(url)

        if r.status_code !=200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)

    except Exception as e:
        print("접속에 실패했습니다.")
        print(e)  

        break

    r.encoding = "utf-8"
    mydict = json.loads(r.text)

    mydata += mydict['documents']
    print("%d 페이지로부터 %d건의 데이터를 수집했습니다." % (page,len(mydict['documents'])))

    isEnd = mydict['meta']['is_end']

    page += 1

print("fin :)")

1 페이지로부터 50건의 데이터를 수집했습니다.
2 페이지로부터 50건의 데이터를 수집했습니다.
3 페이지로부터 50건의 데이터를 수집했습니다.
4 페이지로부터 50건의 데이터를 수집했습니다.
5 페이지로부터 50건의 데이터를 수집했습니다.
6 페이지로부터 50건의 데이터를 수집했습니다.
7 페이지로부터 50건의 데이터를 수집했습니다.
8 페이지로부터 50건의 데이터를 수집했습니다.
9 페이지로부터 50건의 데이터를 수집했습니다.
10 페이지로부터 50건의 데이터를 수집했습니다.
11 페이지로부터 50건의 데이터를 수집했습니다.
12 페이지로부터 50건의 데이터를 수집했습니다.
13 페이지로부터 50건의 데이터를 수집했습니다.
14 페이지로부터 50건의 데이터를 수집했습니다.
15 페이지로부터 50건의 데이터를 수집했습니다.
16 페이지로부터 50건의 데이터를 수집했습니다.
17 페이지로부터 50건의 데이터를 수집했습니다.
18 페이지로부터 50건의 데이터를 수집했습니다.
19 페이지로부터 50건의 데이터를 수집했습니다.
20 페이지로부터 50건의 데이터를 수집했습니다.
21 페이지로부터 50건의 데이터를 수집했습니다.
22 페이지로부터 50건의 데이터를 수집했습니다.
23 페이지로부터 50건의 데이터를 수집했습니다.
24 페이지로부터 50건의 데이터를 수집했습니다.
25 페이지로부터 50건의 데이터를 수집했습니다.
26 페이지로부터 50건의 데이터를 수집했습니다.
27 페이지로부터 50건의 데이터를 수집했습니다.
28 페이지로부터 50건의 데이터를 수집했습니다.
29 페이지로부터 50건의 데이터를 수집했습니다.
30 페이지로부터 50건의 데이터를 수집했습니다.
31 페이지로부터 50건의 데이터를 수집했습니다.
32 페이지로부터 50건의 데이터를 수집했습니다.
33 페이지로부터 50건의 데이터를 수집했습니다.
34 페이지로부터 50건의 데이터를 수집했습니다.
35 페이지로부터 50건의 데이터를 수집했습니다.
36 페이지로부터 50건의 데이터를 수집했습니다.
3

In [15]:
with open("%s_이미지_검색결과.csv" % query, "w", encoding="utf-8") as f:
    f.write("컬렉션,출처, 가로, 세로, 문서URL, 미리보기URL, 이미지URL\n")

    for i, v in enumerate(mydata):
        collection, thumbnail_url, image_url, width, height, display_sitename, doc_url, datetime=v.values()
        f.write("%s,%s,%s,%s,%s,%s,%s\n" % (collection, display_sitename, width, height, doc_url, thumbnail_url, image_url))

print("fin :)")

fin :)


In [16]:
print("다운로드 받을 이미지 수: %d" % len(mydata))

다운로드 받을 이미지 수: 2147


In [26]:
downloadDir = "%s_%s" % (query, dt.datetime.now().strftime("%y%m%d_%H%M%S"))
print(downloadDir)

if not os.path.exists(downloadDir):
    os.mkdir(downloadDir)

python_231211_114639


In [27]:
def kakaoImageDownload(session, docUrl, imageUrl, savePath):
    session = requests.Session()
    session.headers.update({
        "Referer": docUrl,
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    })

    try:
        r = session.get(imageUrl, stream=True)

        if r.status_code !=200:
            msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
            raise Exception(msg)

    except Exception as e:
       print("접속에 실패했습니다.")
       print(e)  

    r.encoding = 'utf-8'

    with open(savePath, "wb") as f:
        f.write(r.raw.read())
        print("%s 파일이 저장되었습니다." % savePath)
    
    session.close()

In [29]:
# 동기식 호출

#max = len(mydata)
max = 10

progress = tqdm(total=max)

for i, v in enumerate(mydata):
    kakaoImageDownload(v['doc_url'], v['image_url'], "%s/%05d.png" % (downloadDir, i))
    progress.update()
    
   
    if i > max:
        break

python_231211_114639/00000.png 파일이 저장되었습니다.
python_231211_114639/00001.png 파일이 저장되었습니다.
python_231211_114639/00002.png 파일이 저장되었습니다.
python_231211_114639/00003.png 파일이 저장되었습니다.
python_231211_114639/00004.png 파일이 저장되었습니다.
python_231211_114639/00005.png 파일이 저장되었습니다.
python_231211_114639/00006.png 파일이 저장되었습니다.
python_231211_114639/00007.png 파일이 저장되었습니다.
python_231211_114639/00008.png 파일이 저장되었습니다.
python_231211_114639/00009.png 파일이 저장되었습니다.
python_231211_114639/00010.png 파일이 저장되었습니다.
python_231211_114639/00011.png 파일이 저장되었습니다.
python_231211_114639/00012.png 파일이 저장되었습니다.
python_231211_114639/00013.png 파일이 저장되었습니다.
python_231211_114639/00014.png 파일이 저장되었습니다.
python_231211_114639/00015.png 파일이 저장되었습니다.
python_231211_114639/00016.png 파일이 저장되었습니다.
python_231211_114639/00017.png 파일이 저장되었습니다.
python_231211_114639/00018.png 파일이 저장되었습니다.
python_231211_114639/00019.png 파일이 저장되었습니다.
python_231211_114639/00020.png 파일이 저장되었습니다.
python_231211_114639/00021.png 파일이 저장되었습니다.
python_231211_114639/00022.png 파

KeyboardInterrupt: 

In [30]:
# 비동기식 호출

downloadDir = "%s_%s" % (query, dt.datetime.now().strftime("%y%m%d_%H%M%S"))
print(downloadDir)

if not os.path.exists(downloadDir):
    os.mkdir(downloadDir)

python_231211_115223


In [31]:
max = len(mydata)
max=50
with futures.ThreadPoolExecutor(max_workers=10) as executor:
    for i, v in enumerate(mydata):
        fu = executor.submit(kakaoImageDownload, v['doc_url'], v['image_url'], "%s/%05d.png" % (downloadDir, i))
        fu.add_done_callback(lambda x : progress.update())
        
        if i > max:
            break

python_231211_115223/00003.png 파일이 저장되었습니다.
python_231211_115223/00006.png 파일이 저장되었습니다.
python_231211_115223/00009.png 파일이 저장되었습니다.
python_231211_115223/00002.png 파일이 저장되었습니다.
python_231211_115223/00022.png 파일이 저장되었습니다.
python_231211_115223/00004.png 파일이 저장되었습니다.
python_231211_115223/00007.png 파일이 저장되었습니다.
python_231211_115223/00023.png 파일이 저장되었습니다.
python_231211_115223/00000.png 파일이 저장되었습니다.
python_231211_115223/00012.png 파일이 저장되었습니다.
python_231211_115223/00010.png 파일이 저장되었습니다.
python_231211_115223/00025.png 파일이 저장되었습니다.
python_231211_115223/00024.png 파일이 저장되었습니다.
python_231211_115223/00013.png 파일이 저장되었습니다.
python_231211_115223/00027.png 파일이 저장되었습니다.
python_231211_115223/00030.png 파일이 저장되었습니다.
python_231211_115223/00028.png 파일이 저장되었습니다.
python_231211_115223/00032.png 파일이 저장되었습니다.
python_231211_115223/00037.png 파일이 저장되었습니다.
python_231211_115223/00005.png 파일이 저장되었습니다.
python_231211_115223/00033.png 파일이 저장되었습니다.
python_231211_115223/00031.png 파일이 저장되었습니다.
python_231211_115223/00001.png 파